In [1]:
# system imports
import os
import time

In [2]:
# model imports
# All torch related imports
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch import nn, optim
import torchvision

In [3]:
from torch.nn import Softmax

In [4]:
# visualization imports
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np

In [5]:
# metrics imports
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

In [6]:
from tqdm.notebook import tqdm as tq

In [7]:
IMAGE_DIMS = 28

In [8]:
# pytorch device configurations
BATCH_SIZE = 512
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
TEST = pd.read_pickle('./sync_pickles/testing_set.pkl')

In [10]:
TEST

,labels,images
0,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...
8515,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8516,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8517,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
8518,170,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."


In [37]:
class MetricsCalculator:
    def __init__(self):

        self.accuracy = 0
        self.loss = 0
        self.macro_f1_score = 0
        self.accuracy_history = []
        self.loss_history = []
        self.macro_f1_history = []

        self.all_preds = torch.tensor([])
        self.top_preds = torch.tensor([])
        self.truth = torch.tensor([])

    def compute_scores(self, preds_tensor, labels_tensor, loss):
        softmax = Softmax(dim=1)
        preds_tensor = softmax(preds_tensor)
        
        values, max_indices = torch.max(preds_tensor, dim=1)

        preds = max_indices.detach().cpu().numpy()
        truth = labels_tensor.detach().cpu().numpy()

        accuracy = accuracy_score(truth, preds)
        macro_f1 = f1_score(truth, preds, average="macro")
        
        self.accuracy += accuracy
        self.macro_f1_score += macro_f1
        self.loss += loss.detach().cpu()
        
        return accuracy, macro_f1

    def save_predictions(self, preds, labels, index):
        softmax = Softmax(dim=1)
        
        preds = preds.detach().cpu()
        labels = labels.detach().cpu()
        
        preds_tensor = softmax(preds)
        values, max_indices = torch.max(preds_tensor, dim=1)
        
        results = torch.topk(preds_tensor, 3)
        self.all_preds = torch.cat((self.all_preds, max_indices), dim=0)
        #self.top_preds = torch.cat((self.top_preds, results), dim=0)
        print(type(results))
        self.truth = torch.cat((self.truth, labels), dim=0)

In [38]:
class LeanIsolatedCharacterDataset(Dataset):
    def __init__(self, x, y, transforms=None):
        ### labels
        self.labels = y.to_numpy()

        ### loading dataset into memory
        self.images = x.to_numpy()

        ### transformations to apply on images
        self.transforms = transforms

    def __getitem__(self, index):
        # convert labels to tensor
        label = torch.tensor(self.labels[index])
        # load single image from list of all preloaded images
        image = self.images[index]
        if self.transforms:
            ## apply transforms
            image = self.transforms(image)
            image = image.float()
        label = label.long()
        return image, label, index

    def __len__(self):
        return len(self.images)

In [39]:
DATA_NORMALIZER = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),transforms.Resize((IMAGE_DIMS,IMAGE_DIMS))])

In [40]:
TESTER = LeanIsolatedCharacterDataset(TEST.images, TEST.labels, DATA_NORMALIZER)

In [41]:
TESTING_LOADER = DataLoader( dataset=TESTER, batch_size=BATCH_SIZE , shuffle=False )

In [42]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    # Identity block
    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)
        # x = self.relu(x) #custom

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [43]:
class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels change
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [44]:
def ResNet50(img_channel=3, num_classes=171):
    return ResNet(block, [2, 3, 5, 2], img_channel, num_classes)
net = ResNet50()

In [45]:
net.load_state_dict(torch.load('./completed/model_1/custom_resnet_50_minimum_validation_loss.pth'))

<All keys matched successfully>

In [46]:
optimizer = optim.Adam(net.parameters(), lr=0.07)  # learning rate
# defining the loss function
criterion = nn.CrossEntropyLoss()  # reduction='none'
net = net.to(DEVICE)
criterion = criterion.to(DEVICE)

In [47]:
def test(model,data_loader):
    model.eval()
    print('testing')
    metrics = MetricsCalculator()
    
    for i,items in tq(enumerate(data_loader, 0)):
        inputs, labels, indexes = items
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        with torch.no_grad():
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            metrics.compute_scores(preds_tensor=outputs, labels_tensor=labels, loss=loss)
            
            # saving predictions 
            metrics.save_predictions(outputs, labels, indexes)
    
    print(len(data_loader))
    
    metrics.accuracy = metrics.accuracy /len(data_loader)
    metrics.loss = metrics.loss /len(data_loader)
    metrics.macro_f1_score = metrics.macro_f1_score/ len(data_loader)
    
    return metrics            

In [48]:
results = test(net,TESTING_LOADER)

testing


<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>
<class 'torch.return_types.topk'>

17


In [107]:
cm = confusion_matrix(results.truth.numpy().astype(int), results.all_preds.numpy().astype(int))

In [108]:
results.all_preds.shape

torch.Size([8520])

In [109]:
len(cm)

171

In [110]:
results.accuracy

0.8031014616212339

In [111]:
results.macro_f1_score

0.17206946733681894

In [112]:
results.truth.numpy()

array([  0.,   0.,   0., ..., 170., 170., 170.], dtype=float32)

In [113]:
results.all_preds.numpy()

array([  0., 126.,   0., ..., 170., 170., 170.], dtype=float32)

In [114]:
LABEL = [x for x in range(171)]

In [140]:
report = classification_report(results.truth.numpy(), results.all_preds.numpy(), labels=LABEL,output_dict=True) #output_dict=True

In [141]:
cm

array([[38,  0,  0, ...,  0,  0,  0],
       [ 0, 33,  0, ...,  0,  0,  0],
       [ 0,  0, 36, ...,  0,  1,  0],
       ...,
       [ 0,  0,  0, ..., 48,  0,  0],
       [ 0,  0,  0, ...,  0, 50,  0],
       [ 0,  0,  0, ...,  0,  0, 40]], dtype=int64)

In [142]:
from sklearn.metrics import plot_confusion_matrix as plotter

In [143]:
cm_df = pd.DataFrame(cm)

In [144]:
cm_df

,0,1,2,3,4,5,6,7,8,9,...,161,162,163,164,165,166,167,168,169,170
0,38,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,33,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,47,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,49,0,0,0,0
167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,44,0,0,0
168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,48,0,0
169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,50,0


In [145]:
cm_df.to_html('./results/custom_restnet_28x28/custom_restnet_28x28_regular.html')

In [146]:
len(report)

174

In [147]:
report

{'0': {'precision': 0.8837209302325582,
  'recall': 0.7307692307692307,
  'f1-score': 0.8,
  'support': 52},
 '1': {'precision': 0.9166666666666666,
  'recall': 0.7674418604651163,
  'f1-score': 0.8354430379746837,
  'support': 43},
 '2': {'precision': 0.8571428571428571,
  'recall': 0.7346938775510204,
  'f1-score': 0.7912087912087913,
  'support': 49},
 '3': {'precision': 0.7741935483870968,
  'recall': 0.9056603773584906,
  'f1-score': 0.8347826086956522,
  'support': 53},
 '4': {'precision': 0.7580645161290323,
  'recall': 0.9215686274509803,
  'f1-score': 0.831858407079646,
  'support': 51},
 '5': {'precision': 0.5294117647058824,
  'recall': 0.9183673469387755,
  'f1-score': 0.6716417910447762,
  'support': 49},
 '6': {'precision': 0.7258064516129032,
  'recall': 0.8823529411764706,
  'f1-score': 0.7964601769911503,
  'support': 51},
 '7': {'precision': 0.8648648648648649,
  'recall': 0.6530612244897959,
  'f1-score': 0.7441860465116279,
  'support': 49},
 '8': {'precision': 0.79